In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from pathlib import Path

# Small style adjustments for more readable plots
plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

In [27]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import fsspec

import xarray as xr
xr.set_options(display_style="html")  # display dataset nicely

import warnings
warnings.simplefilter("ignore")  # filter some warning messages

# code features from https://scitools.org.uk/cartopy/docs/v0.14/matplotlib/feature_interface.html
crs = ccrs.PlateCarree()  # set projection

In [36]:
# load dataset
ds = xr.open_dataset("~/shared/climate-data/ds_hw.nc")
# subset dataset for west coast
ds_sst_loc = ds.analysed_sst.sel(lat=slice(35, 45), lon=slice(-130, -120))

# get avg climatology for 2002-2012
ref_sst = ds_sst_loc.sel(time=slice("2002-01", "2012-12")).mean()
ref_sst

<xarray.DataArray 'analysed_sst' ()>
array(287.07602, dtype=float32)

In [37]:
# get sst stdev between 2002 and 2012
ref_sst_std = ds_sst_loc.sel(time=slice("2002-01", "2012-12")).std("time")
ref_sst_std

<xarray.DataArray 'analysed_sst' (lat: 167, lon: 167)>
array([[2.1389308, 2.1354287, 2.1338828, ...,       nan,       nan,
              nan],
       [2.1375268, 2.1377196, 2.134783 , ...,       nan,       nan,
              nan],
       [2.134656 , 2.1342168, 2.1303525, ...,       nan,       nan,
              nan],
       ...,
       [3.0511901, 3.055246 , 3.0596726, ...,       nan,       nan,
              nan],
       [3.0519376, 3.055465 , 3.0564735, ...,       nan,       nan,
              nan],
       [3.0555851, 3.0567126, 3.05867  , ...,       nan,       nan,
              nan]], dtype=float32)
Coordinates:
  * lat      (lat) float32 35.04 35.1 35.16 35.22 ... 44.82 44.88 44.94 45.0
  * lon      (lon) float32 -130.0 -129.9 -129.8 -129.8 ... -120.1 -120.1 -120.0

In [38]:
def to_monthly(ds):
    """
    Supposedly groups ds by year and month...
    """
    year = ds.time.dt.year
    month = ds.time.dt.month

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), month=("time", month.data))

    # reshape the array to (..., "month", "year")
    return ds.set_index(time=("year", "month")).unstack("time")  

In [46]:
# get dataset for fig1 timeframe (2014-2016)
ds_fig1 = ds_sst_loc.sel(time=slice("2014-01", "2016-09"))
# subtract 2002-2012 average
ds_fig1_norm = ds_fig1 - ref_sst

# group by month and year
ds_fig1_grouped = to_monthly(ds_fig1_norm)
ds_fig1_grouped.max()

<xarray.DataArray 'analysed_sst' ()>
array(7.689972, dtype=float32)